# Lab 02 - Introduction to NLP

## The dataset

### 1. How many splits does the dataset has ?

In [ ]:
from datasets import get_dataset_split_names

get_dataset_split_names("imdb")

There is three splits: train, test and unsupervised.

### 2. How big are these splits ?

In [ ]:
from datasets import load_dataset

dataset = load_dataset("imdb")
dataset

In [ ]:
dataset.num_rows

The train set has 25 000 rows, as the test set and the unsupervised has 50 000 rows.

### 3. What is the proportion of each class on the supervised splits ?

In [ ]:
dataset.num_columns

In [ ]:
dataset.column_names

In [ ]:
dataset.data['train']['label'].value_counts()

In [ ]:
dataset.data['test']['label'].value_counts()

The test and train sets are composed of half label 0 and half label 1.

## Naive Bayes classifier

#### Question 1

In [ ]:
# import local function from ./lib/util.py
from lib.util import lower_and_remove_punctuation

In [ ]:
dataset_transform = dataset.map(lower_and_remove_punctuation)
dataset_transform.data['train']['text'][0]

#### Question 2

We have decided to use scikit-learn Pipeline with a CountVectorizer and MunltinomialNB classifier.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
pipe = make_pipeline(CountVectorizer(), MultinomialNB())

In [ ]:
pipe.fit(dataset_transform.data['train']['text'].to_numpy(), dataset_transform.data['train']['label'].to_numpy())

In [ ]:
# Compare predictions and labels
prediction_train = pipe.predict(dataset_transform.data['train']['text'].to_numpy()) == dataset_transform.data['train']['label'].to_numpy()
prediction_test = pipe.predict(dataset_transform.data['test']['text'].to_numpy()) == dataset_transform.data['test']['label'].to_numpy()

#### Question 3

In [ ]:
prediction_train.sum()/len(prediction_train)

The accuracy on the training set is around 91%.

In [ ]:

prediction_test.sum()/len(prediction_test)

The accuracy on the test set is around 82%.

#### Question 4 : Why is accuracy a sufficient measure of evaluation here ?  

The labels are boolean values, there is no need for a more accurate distance between different predictions.  

#### Question 6

In [ ]:
list_index = [index for index,value in enumerate(prediction_test) if not value] # list of false element by prediction
# len(list_index) / len(prediction_test) check if we have the good amount of false answers

In [ ]:
list_index_0 = [index for index,value in enumerate(dataset_transform.data['test']['label'].to_numpy()) if value == 0 and index in list_index]
list_index_1 = [index for index,value in enumerate(dataset_transform.data['test']['label'].to_numpy()) if value == 1 and index in list_index]

In [ ]:
print("text: ", dataset_transform.data['test']['text'][list_index_0[0]])
print("label: ", dataset_transform.data['test']['label'][list_index_0[0]])

In this example the model has probably failed because the comment is not fully negative, it said that it's 'decent', have some 'problem'...

In [ ]:
print("text: ", dataset_transform.data['test']['text'][list_index_1[0]])
print("label: ", dataset_transform.data['test']['label'][list_index_1[0]])

In this case the model classify wrong a text that was positive. 

## **Stemming** ~~and Lemmatization~~

#### Question 1

We choose stemming pretreatment

In [ ]:
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize 
# We need to download a package for word tokenization
nltk.download('punkt')

In [ ]:
import re

def stemming_builder():
    re_word = re.compile(r"^\w+$")
    stemmer = SnowballStemmer("english")
    def wesh(text):
        stemmed = [stemmer.stem(word) for word in word_tokenize(text['text'].lower()) if re_word.match(word)]
        text['text'] = " ".join(stemmed)
        return text
    return wesh

In [ ]:
dataset_stemming = dataset.map(stemming_builder())
dataset_stemming.data['train']['text'][0]

#### Question 2

In [ ]:
pipe.fit(dataset_stemming.data['train']['text'].to_numpy(), dataset_stemming.data['train']['label'].to_numpy())

In [ ]:
prediction_train = pipe.predict(dataset_stemming.data['train']['text'].to_numpy()) == dataset_stemming.data['train']['label'].to_numpy()
prediction_test = pipe.predict(dataset_stemming.data['test']['text'].to_numpy()) == dataset_stemming.data['test']['label'].to_numpy()

In [ ]:
prediction_train.sum()/len(prediction_train)

In [ ]:
prediction_test.sum()/len(prediction_test)

#### Question 3

Results be like :  
![homer gif](ressources/homer.gif)